[ 자료 출처 ] 네이버 평점 자료

https://github.com/e9t/nsmc


[ 예제 출처 ] 도서 - 파이썬 머신러닝 완벽 가이드


In [24]:
# 데이타 확인

import pandas as pd

train_df = pd.read_csv('nsmc-master/ratings_train.txt', sep='\t')
train_df.head(20)

# label 값이 1이면 긍정, 0이면 부정

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [2]:
# 0과 1 (부정과 긍정)의 비율이 균등한 분포임을 확인
train_df['label'].value_counts()


0    75173
1    74827
Name: label, dtype: int64

In [4]:
# null 값 확인
train_df.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [11]:
# 'document' 컬럼에 Null 값이 있는데 이는 공백으로 변환
train_df = train_df.fillna(' ')

# 숫자를 공백으로 변경
import re
train_df['document'] = train_df['document'].apply(lambda x: re.sub(r"\d+",' ', x))

In [12]:
# 테스트 데이타로 위와 동일한 작업
test_df = pd.read_csv('nsmc-master/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r"\d+",' ', x))

In [13]:
#!pip install konlpy

In [14]:
#!pip install sklearn

In [16]:
# 형태소 분석
# Twitter 객체의 morphs() 는 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 리스트 객체로 반환

from konlpy.tag import Okt

twitter = Okt()

In [17]:
# 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화하여 리스트 형태로 변환
def tw_tokenizer(text):
    tokens_ko = twitter.morphs(text)
    return tokens_ko

# 텍스트 분석

- 텍스트 분석은  비정형 데이타인 텍스트를 벡터값을 가지는 피처로 변환하는 것이 중요하다
- 숫자형의 피처 기반 데이타만 머신러닝 알고리즘이 처리하기 때문에 텍스트를 의미 있는 피처 형태로 추출해야 한다



### [ TF-IDF 참고 ](https://soojle.gitbook.io/project/undefined-2/nlp/tf-idf-term-frequency-inverse-document-frequency)



---

## 사이킷런에서 TF-IDF 벡터화 - TfidfVectorizer 클래스 이용


+ max_df : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
         너무 높은 빈도수를 가지는 단어는 문법적인 특성으로 반복되는 단어일 가능성이 높기에 이를 제거
         max_df=100 같이 정수값이면 전체 문서에서 100개 이하로 나타나는 단어만 피처로 추출한다
         max_df=0.9 같이 실수값이면 전체 문서에 걸쳐 빈도수가 0~95%까지의 단어만 피처로 추출하고 나머지 상위 5%는 추출하지 않는다
         
+ min_df : 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
         너무 낮은 빈도수의 단어는 중요하지 않을 단어일 가능성이 높기에 이를 제거
         mix_df=10 같이 정수값이면 전체 문서에서 10번 이하로 나타나는 단어로 피처로 추출하지 않는다
         mix_df-0.02 같이 실수값이면 전체 문서에 하위 2% 이하의 빈도수를 가지는 단어는 추출하지 않는다

+ max_features : 추출하는 피처의 개수를 제한
         max_features=2000 이라면 가장 높은 빈도수를 가지는 단어 순으로 정렬해 2000개 까지만 추출
         
+ stop_words : 'english'로 지정하면 영어의 스톱워드로 지정된 단어는 추출에서 제외
         stop_words 는 분석에 큰 의미가 없는 단어를 지칭

+ n_gram_range : 예를 들어 (1,2)로 지정하면 토큰화된 단어를 1개(minimun) 그리고 순서대로 2개씩(maximun) 묶어서 피처로 추출한다


+ tokenizer : 토큰화를 별도로 커스텀 함수 이용시 지정



### * 텍스트 데이타의 피처 벡터화 변환

In [23]:
# 시간소요 ( 노트북 16G : 30 분이상)
# TF-IDF 방식으로 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
# 상위 90%로 제한
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])


In [26]:

# 학습 데이타에 적용한 TfidfVectorizer를 테스트 데이터에도 사용 ( 시간소요 : 10분)
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

### * 회귀를 이용해 감성 분석 분류 ( 학습/예측/평가수행)

In [ ]:

lg_clf = LogisticRegression(random_state=0)

# -------------- 모델에 학습
lg_clf.fit(tfidf_matrix_train, train_df['label'])

# -------------- 테스트를 예측
preds = lg_clf.predict(tfidf_matrix_test)

# --------------- 정확도
from sklearn.metrics import accuracy_score
print('TF-IDF LogisticRegression의 예측 정확도 : ', accuracy_score(test_df['label'], preds))

### GridSearchCV를 이용하여 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행

In [14]:

# 파라미터 C 최적화를 위해 GridSearchCV를 이용
params = {'C':[1, 3.5, 4.5, 5.5, 10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_,4))

# [결과] {'C': 3.5} 0.8592
# C가 3.5일 때 0.8592 정확도임

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\simmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\simmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:

{'C': 3.5} 0.8592


C:\Users\simmi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
# classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

from sklearn.metrics import accuracy_score
print('LogisticRegression 정확도: ', accuracy_score(test_df['label'], preds))

LogisticRegression 정확도:  0.86184


# 댓글로 부정인지 긍정인지 확인

In [25]:
# type(test_df['document']) -> Series
comment = pd.Series(['안보면 후회'])
tfidf_matrix_test = tfidf_vect.transform(comment)
preds = best_estimator.predict(tfidf_matrix_test)
preds

#부정 0 , 긍정 1

NameError: name 'best_estimator' is not defined

In [ ]:
comment = pd.Series(['감독 꼬라지하고는'])
tfidf_matrix_test = tfidf_vect.transform(comment)
preds = best_estimator.predict(tfidf_matrix_test)
preds

In [ ]:
comment = pd.Series(['이 훌륭한 배우들을 데려다가'])
tfidf_matrix_test = tfidf_vect.transform(comment)
preds = best_estimator.predict(tfidf_matrix_test)
preds

In [ ]:
comment = pd.Series(['이 훌륭한 배우들을 데려다가 이렇게 만들었어'])
tfidf_matrix_test = tfidf_vect.transform(comment)
preds = best_estimator.predict(tfidf_matrix_test)
preds

In [ ]:
comment = pd.Series(['잘하는짓이다'])
tfidf_matrix_test = tfidf_vect.transform(comment)
preds = best_estimator.predict(tfidf_matrix_test)
preds